In [1]:

from grading_tools import *
import ipywidgets as widgets
import os
from IPython.display import display
%matplotlib widget

# Uncertainty Analysis
This tutorial shows how to use CoMETS to perform an uncertainty analysis with the Brewery model. This type of experimentation aims at studying and quantifying how the uncertainty on inputs of a model propagates through the model and leads to variability in the outputs of this model.

## The Brewery model

Like in the previous tutorial, we are working with the Brewery model. It is a drink consumption model including stock level management and customer satisfaction dynamics. The model input parameters that we will deal with are: 

- `NbWaiters`: number of waiters in the bar
- `RestockQty`: a fix quantity of drinks added to the current stock of the bar once it reaches a threshold value.

The Brewery model has no default outputs as the value of all attributes at the end of a simulation can be considered as an output. In our case we will use the following attributes:

 - `AverageSatisfaction`: average customer satisfaction
 - `Stock`: stock of the bar.


For this tutorial, we suppose that the model's inputs (i.e `NbWaiters` and `RestockQty`) are uncertain. The model remains deterministic. We want to answer the following question: **How are the average customer satisfaction and the stock affected by the uncertainty on the model's inputs?** 

## The approach
An uncertainty analysis requires to specify precisely what is the task we are experimenting with: what are its inputs and ouputs? In our case the inputs are `NbWaiters` and `RestockQty`, while the outputs are the `AverageSatisfaction` and `Stock`. Along with that we need to precisely specify the uncertainty on the input variables. In this study, we will model the uncertainty on the inputs with two **uniform discrete distributions**. The uncertainty analysis will then return the default statistics for the variability of the `AverageSatisfaction` and the `Stock`: mean, standard deviation, standard error of the mean, 95% confidence interval of the estimated mean. These results will be used to answer the question raised in the problem statement.

## 1. Define the task on which to experiment
CoMETS always works with the same method: it performs analyses on tasks. Before defining any experiment we need to define the task on which it will be performed. A task behaves as a function that takes as input a ParameterSet and outputs another ParameterSet. It has a method `evaluate()` that is in charge of performing the evaluation and returning the outputs.
During the task evaluation, the model is run. When creating the task, we also allow for some additional pre-processing (i.e encoding) of the input parameters or post-processing of the outputs of the model.



In [2]:
# code to display the diagram below
from IPython.display import Image
Image(url= "Comets_ModelTask.png", width=1000, height=1600)

To run the analysis we first need to import comets:

In [3]:
import comets 

In order to import the Brewery model, we then need to specify the path to the project containing it.

In [4]:
#This example assumes that this notebook is in the folder containing the Brewery project
#and that the model has been compiled with the python wrappers activated.

from pathlib import Path
cwd = Path().resolve()

### Import the simulator
We need to import the simulator in order to allow CoMETS to interact with it. In this example, we work with the Brewery simulator, which we can instantiate using a `CosmoInterface` as follows:

In [5]:
simulator = comets.CosmoInterface(
    simulator_path = 'BreweryTutorialSimulation',
    project_path = cwd,
)

For more details on how to instantiate the Brewery model, please see the tutorial "Running simulations with CoMETS".

### Define the input parameters of the task and the encoding for working with the model
The task should take as input a ParameterSet, which is a set of parameters, stored as a dictionary containing names of parameters and their values. In our case the input parameters are the NbWaiters and the RestockQty, which would take the form 
`{'NbWaiters': NbWaiters, 'RestockQty': RestockQty}`.



In order for the model to recognize input parameters, we need to encode them to a ParameterSet where the keys are the datapaths of the corresponding attributes in the model. For NbWaiters and RestockQty it corresponds to: 

- `Model::{Entity}MyBar::@NbWaiters`
- `Model::{Entity}MyBar::@RestockQty`

The encoding function takes the ParameterSet:

`{'NbWaiters': NbWaiters, 'RestockQty': RestockQty}`

and returns the ParameterSet:

`{'Model::{Entity}MyBar::@NbWaiters': NbWaiters, 'Model::{Entity}MyBar::@RestockQty': RestockQty}`.

In [6]:
def encoder(parameters):
    return {'Model::{Entity}MyBar::@NbWaiters': parameters['NbWaiters'],
            'Model::{Entity}MyBar::@RestockQty': parameters['RestockQty']}
    

### Define the quantities of interest for the experiment


The quantities we are interested in are the AverageSatisfaction and the Stock, which are the outputs of the task. The task should return a ParameterSet with the values of these quantities. The AverageSatisfaction and the Stock are available in the model via their respective datapath :


- `Model::{Entity}MyBar::@AverageSatisfaction`
- `Model::{Entity}MyBar::@Stock`.

The quantities of interest can be computed from the outputs of the model by creating a function that takes as input the CosmoInterface and returns a ParameterSet with the values of the quantities of interest.

In [7]:
def get_outcomes(simulator):
    outputs = simulator.get_outputs(
        ['Model::{Entity}MyBar::@AverageSatisfaction', 'Model::{Entity}MyBar::@Stock'])
    AverageSatisfaction = outputs['Model::{Entity}MyBar::@AverageSatisfaction']
    stock = outputs['Model::{Entity}MyBar::@Stock']
    return {'AverageSatisfaction' : AverageSatisfaction, 'Stock' : stock}

### Declaring the Task

Once the inputs and outputs have been defined, the final step is to declare the task with its `CosmoInterface`, `encode` and `get_outcomes` method. Evaluating the task corresponds to encoding the inputs, running a simulation and then computing the quantity of interest with the `get_outcomes` function.

In [8]:
uncertaintytask = comets.ModelTask(
    simulator, 
    get_outcomes = get_outcomes, 
    encode = encoder
)

## 2. Model the uncertainty on the inputs

Once the task has been defined, we need to choose how to represent the uncertainty on its inputs. This can be done either by declaring a generator or declaring a distribution. In this tutorial, we will only treat the case where the uncertainty is modeled by distributions. We define the function to generate samples with a list of dictionaries, one for each independent parameter. Each dictionary contains the following information:

- `name` (str): name of the parameter, should correspond to the name of a task input parameter
- `sampling` (str): name of the probability distribution for this parameter
- `parameters` (dict): parameters of the distribution, in the form of a dictionary with named parameters and their values
- `size` (optional, int): size of the list if the parameter corresponds to a list of independent and identically distributed (i.i.d.) random variables.

The currently available distributions and their parameters are listed in the `DistributionRegistry`, accessible with: `print(comets.DistributionRegistry)`. Here is a non-exhaustive list of distributions we support: 

- `normal`: its parameters are {`loc`: mean of the distribution, `scale`: standard deviation}
- `exponential`: its parameters are {`loc`: location of the lower bound, `scale`: scale parameter, }
- `uniform`: its parameters are {`loc`: lower threshold of distribution, `scale`: upper threshold of distribution}
- `discreteuniform`: its parameters are {`low`: lower threshold of distribution, `high`: upper threshold of distribution}

In the Brewery model, the `NbWaiters` and the `RestockQty` are represented by integers. We will model them with two uniform discrete distributions. The NbWaiters' distribution will range from 1 to 4 (included) and the RestockQty's distribution will range from 5 to 15 (included). 

In [9]:
sampling = [
    {
        "name": 'NbWaiters',
        "sampling": "discreteuniform",
        "parameters": {'low': 1, 'high': 5},
    },
    {
        "name": 'RestockQty',
        "sampling": "discreteuniform",
        "parameters": {'low': 5, 'high': 16},
    }
]

## 3. Declaring the Experiment
An uncertainty analysis is an experiment taking the following arguments: 
- `task`: task on which to perform the analysis
- `sampling`: defines the function to generate samples on the input parameters. It consists in a list of distributions or generators.
- `method` (optional, default to “random”): string, defines the sequence used to generate the samples. “random” defines pseudo-random number generation, resulting in a Monte Carlo integration method.
- `analyzer` (optional, default to “standard”): string or list of strings, specifies the list of output statistics computed at the end of the experiment. For example, “standard” computes the mean, standard deviation, standard error of the mean, and 95% confidence interval of the mean.
- `n_jobs`: number of processes used for parallel computing. Defaults to 1 (no parallelization).
- `stop_criteria`: dictionary containing the stop criteria of the experiment. In our case we use ‘max_evaluations’, which corresponds to the number of task evaluations to perform. 

To know more about available distributions, methods, analyzers and stop criteria available, refer to CoMETS' documentation. 


In [10]:
ua = comets.UncertaintyAnalysis(
        task = uncertaintytask,
        sampling = sampling,
        n_jobs = 6,
        stop_criteria = {'max_evaluations' : 1000})

## 4. Running the experiment

In [11]:
ua.run()

## 5. Displaying the results of the Experiment
The results of the analysis are stored in the attribute 'results', it contains default statistics on the quantity of interest (the service level).

In [12]:
ua.results["statistics"]

,mean,std,sem,confidence interval of the mean at 95%
AverageSatisfaction,0.880240,0.439813,0.013894,"(0.852974392990877, 0.9075046489252908)"
Stock,14.333333,3.259761,0.102980,"(14.131252370437151, 14.535414296229517)"


It is also possible to access all the results of evaluations of the task in order to compute other statistics. Below we display the first five evaluation results.

In [13]:
ua.list_of_results[:5]

[{'AverageSatisfaction': 1, 'Stock': 14},
 {'AverageSatisfaction': 1, 'Stock': 14},
 {'AverageSatisfaction': 0, 'Stock': 15},
 {'AverageSatisfaction': 0, 'Stock': 16},
 {'AverageSatisfaction': 1, 'Stock': 13}]

## Question 1

 <span style="font-size: larger;"> What is the purpose of `sampling` input argument of UncertaintyAnalysis ? </div>

**1** - It defines the output variables of the simulation on which we want to compute uncertainties. 

**2** - It describes how the input samples of the uncertainty analysis will be generated.

**3** - It specifies which Monte-Carlo algorithm is used to run the different simulations.

**4** - 

In [14]:
#
question(T3Q1)

## Question 2

 <span style="font-size: larger;"> Which argument should I use to specify the number of simulation to perform ? </div>

**1** - `n_jobs`

**2** - `batch_size`

**3** - the'max_evaluations' key of `stop_criteria`

**4** - `number_of_simulation_to_perform`

In [15]:
#
question(T3Q2)